In [1]:
# To Mount Drive
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/Project/")

Mounted at /content/drive


In [ ]:
!pip install gradio tensorboardX && clear

In [3]:
from io import BytesIO
from torch import argmax, load
from torch import device as DEVICE
from torch.cuda import is_available
from torch.nn import Sequential, Linear, SELU, Dropout, LogSigmoid
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Resize, functional, Grayscale
from torchvision.models import resnet50
from bts.model import DynamicUNet
from bts.classifier import BrainTumorClassifier
import numpy as np
import gradio as gr
import cv2

In [4]:
LABELS = ['None', 'Meningioma', 'Glioma', 'Pitutary']

In [5]:
device = DEVICE("cuda:0" if is_available() else "cpu")

# LOAD THE RESNET MODEL

In [ ]:
resnet_model = resnet50(pretrained=True)

for param in resnet_model.parameters():
    param.requires_grad = True

n_inputs = resnet_model.fc.in_features
resnet_model.fc = Sequential(Linear(n_inputs, 2048),
                            SELU(),
                            Dropout(p=0.4),
                            Linear(2048, 2048),
                            SELU(),
                            Dropout(p=0.4),
                            Linear(2048, 4),
                            LogSigmoid())

for name, child in resnet_model.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

resnet_model.to(device)
resnet_model.load_state_dict(load('/content/drive/MyDrive/Colab Notebooks/Project/models/bt_resnet50_model.pt', map_location=device))
resnet_model.eval()

In [7]:
def resnet_preprocess_image(image_bytes):
  transform = Compose([Resize((512, 512)), ToTensor()])
  img = Image.open(BytesIO(image_bytes))
  return transform(img).unsqueeze(0)

In [8]:
def resnet_get_prediction(image_bytes):
    tensor = resnet_preprocess_image(image_bytes=image_bytes)
    y_hat = resnet_model(tensor.to(device))
    class_id = argmax(y_hat.data, dim=1)
    return str(int(class_id)), LABELS[int(class_id)]

In [9]:
def resnet_classify_image(filepath):
    with open(filepath, 'rb') as file:
        img_bytes = file.read()
    class_id, class_name = resnet_get_prediction(img_bytes)
    return class_name

In [10]:
filter_list = [16, 32, 64, 128, 256]

unet_model = DynamicUNet(filter_list).to(device)
classifier = BrainTumorClassifier(unet_model, device)
model_path = "/content/drive/MyDrive/Colab Notebooks/Project/models/bt_resunet_model.pt"
classifier.restore_model(model_path)

print(f'Saved model at location "{model_path}" loaded on {device}')

Saved model at location "/content/drive/MyDrive/Colab Notebooks/Project/models/bt_resunet_model.pt" loaded on cuda:0


In [11]:
def resunet_get_model_output(image):
    """Returns the saved model output"""
    image = image.view((-1, 1, 512, 512)).to(device)
    output = unet_model(image).detach().cpu()
    output = (output > 0.5)
    output = output.numpy()
    output = np.resize((output * 255), (512, 512))
    return output

In [12]:
def resunet_get_file(filepath):
    """Load the image by taking file name as input"""
    default_transformation = Compose([
        Grayscale(),
        Resize((512, 512))
    ])

    image = default_transformation((Image.open(filepath).resize((512, 512))))
    return functional.to_tensor(image)

In [13]:
def resunet_get_masked_image(filepath):
    image = resunet_get_file(filepath)
    masked_image = resunet_get_model_output(image)
    return masked_image

In [14]:
def construct_segment(filepath, masked_image):
    image = Image.open(filepath)
    original_image = np.asarray(image)
    color = np.array([255,0,0], dtype='uint8')
    numpy_mask_image = np.where(masked_image[...,None], color, original_image)
    segment_image = cv2.addWeighted(original_image, 0.8, numpy_mask_image, 0.2, 0)

    masked_image = masked_image.astype(np.uint8)

    contours = cv2.findContours(masked_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
    contoured_image = cv2.drawContours(segment_image, contours, -1, (255, 0, 0), 2)

    return contoured_image

# Gradio Interface

In [15]:
def predict_image(filepath):
    class_name = resnet_classify_image(filepath)
    masked_image = resunet_get_masked_image(filepath)
    if not np.any(masked_image):
        class_name = "No Tumor"
    segmented_image = construct_segment(filepath, masked_image)
    return (segmented_image, class_name)

In [16]:
interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type="filepath"),
                         outputs=[gr.Image(), gr.Label()],)
                        #  outputs="textbox")

interface.launch(share=True,
                 debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://99b489444bb72aa92c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://99b489444bb72aa92c.gradio.live
